In [7]:
from pickle import load
import pandas as pd
import numpy as np
from api_keys import g_key

import gmaps

import requests

In [8]:
model = load(open('model_artifacts/model.pkl', 'rb'))
scalar = load(open('model_artifacts/scalar.pkl', 'rb'))

bonus_model = load(open('model_artifacts/bonus_model.pkl', 'rb'))
bonus_scalar = load(open('model_artifacts/bonus_scalar.pkl', 'rb'))

equity_model = load(open('model_artifacts/equity_model.pkl', 'rb'))
equity_scalar = load(open('model_artifacts/equity_scalar.pkl', 'rb'))


/Users/iSabyasachi/opt/anaconda3/envs/Python3-8/lib/python3.8/site-packages/sklearn/base.py:324: UserWarning: Trying to unpickle estimator MLPRegressor from version 0.24.1 when using version 1.0.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/iSabyasachi/opt/anaconda3/envs/Python3-8/lib/python3.8/site-packages/sklearn/base.py:324: UserWarning: Trying to unpickle estimator RandomizedSearchCV from version 0.24.1 when using version 1.0.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/iSabyasachi/opt/anaconda3/envs/Python3-8/lib/python3.8/site-packages/sklearn/base.py:324: UserWarning: Trying to unpickle estimator StandardScale

In [9]:
def predict(yac, exp, level, d1, d2, d3):
    X_be=np.array([[level, exp, yac, d1, d2, d3]])
    X_be=X_be.reshape(1, -1)
    bonus_scaled=bonus_scalar.transform(X_be)
    bonus=bonus_model.predict(bonus_scaled)
    
    equity_scaled=equity_scalar.transform(X_be)
    equity=equity_model.predict(equity_scaled)
    
    X=np.array([[level, exp, yac, d1, bonus, equity, d2, d3]])
    X=X.reshape(1, -1)
    scaled=scalar.transform(X)
    salary=model.predict(scaled)
    
    
    Y=[bonus, equity, salary]
    return Y

    
    
    
    
    

In [10]:
def distances(origin_city, destination_cities, g_key):
    base_url = "https://maps.googleapis.com/maps/api/distancematrix/json?"
    distances=[]

    
    for i in range(len(destination_cities)):
        
        url=base_url+"origins="+origin_city+"&destinations="+destination_cities[i]+"%20C"+"&key="+g_key

        payload={}
        headers = {}
        response = requests.request("GET", url, headers=headers, data=payload)

        x=response.json()

        y=x["rows"][0]["elements"][0]["distance"]["value"]

        distances.append(y)
        
    return(distances)


In [12]:
yac=input("Enter years at company(in years): ")
exp=input("Enter years of experience(in years): ")
city=input("Enter company location city: ")
state=input("Enter company location state abbreviation: ")
origin_city=city+"%20C"+state
destination_cities=["New York%20CNY","San Francisco%20CCA", "Seattle%20CWA"]

D_y=distances(origin_city, destination_cities, g_key)

print("Enter your level. Eg. 10 for VP, 1 for Junior for Associate Data Scientist")
level=input("Enter level: ")

result=predict(yac, exp, level, D_y[0], D_y[1], D_y[2])
bonus=result[0]
equity=result[1]
salary=result[2]

print(f"Your expected total yearly compensation is: ${salary}")
print(f"Your expected bonus is: ${bonus}")
print(f"Your expected equity is: ${equity}")

Enter years at company(in years): 0
Enter years of experience(in years): 13
Enter company location city: Carmel
Enter company location state abbreviation: IN
Enter your level. Eg. 10 for VP, 1 for Junior for Associate Data Scientist
Enter level: 8
Your expected total yearly compensation is: $[263.86168468]
Your expected bonus is: $[26.55096361]
Your expected equity is: $[44.78831051]


/var/folders/0_/vd_lh7fd3y1fk1xwqfbgdd_c0000gn/T/ipykernel_73859/1688431811.py:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  X=np.array([[level, exp, yac, d1, bonus, equity, d2, d3]])


In [ ]:
0, 13, Carmel, IN, 8